<img src="img/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

# Sentinel 2 - Indexar una Imágen

**Introducción**

La indexación de imágenes consiste en (1) incarle al cubo de datos dónde tenemos imágenes de un sensor determinado en nuestro sistema de archivos, (2) entregarle al cubo de datos los **metadatos específicos** de la imágen para que este pueda facilitar el proceso de búsqueda, en este proceso también (3) se asocia cada imágen de interés al producto de Sentinel 2 que ya hemos definido con anterioridad.

**Objetivo**

En esta sección indexaremos una imágene del sensor Sentinel 2 en el cubo de datos. Al finalizar verificaremos si la imagen fue indexada correctamente. Podrá repetir el mismo procedimiento para indexar una segúnda imágen.

**Contenido**

1. Descargar imágenes Sentinel 2
2. Descomprimir las imagenes
3. Generar los metadatos de la imágen
4. Adicionar la imágen al índice de imágenes del cubo de datos

## 1. Descargar imágenes Sentinel 2

___

Hasta este punto debría tener 2 imágenes Sentinel 2 descargadas en su computador. Si no es el caso, puede descargar las imágenes [aquí](https://drive.google.com/drive/folders/1eNWlKCcia3FsnhFA7k61m3pnkFp3OiF6?usp=sharing). Para mantener nuestras imágenes organizadas, las pondrémos en el siguiente directorio `/vagrant/indexed_storage/S2_MSI_L2A/`.

## 2. Descomprimir las imagenes

___

Verifique con el comando que se muestra a continaución que las images que descargó anteriormente esten en el directorio `/vagrant/indexed_storage/S2_MSI_L2A/`. Deberá obtener una salida similar a la que se muestra a continaución. Donde se muestran los archivos comprimidos de las imágenes. 

```sh
S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.zip
S2B_MSIL2A_20210115T151709_N0214_R125_T18NYM_20210115T174459.zip
sen2cor_ard.yaml
sen2cor_prepare.py
```

In [ ]:
!ls /vagrant/indexed_storage/S2_MSI_L2A

Seleccione uno de los archivos `.zip` que ha descargado y configure la variable `image_file_name` con el identificador de imagen como se muestra en el siguiente ejemplo.

In [ ]:
# Set image_file_name with the image identifier (do not use the .zip extension)
image_file_name = 'S2B_MSIL2A_20210115T151709_N0214_R125_T18NYM_20210115T174459'

**Use el siguiente comando para descomprimir la imagen seleccionada en el directorio `/vagrant/indexed_storage/S2_MSI_L2A/`**

In [ ]:
image_zip_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.zip'
image_dst_path = '/vagrant/indexed_storage/S2_MSI_L2A/'

!unzip -q -o $image_zip_path -d $image_dst_path

Use el siguiente comando para validar que la imágen a sido descomprimida. La imagen descomprimida quedará en una carpeta con sufijo `.SAFE`, que va presedido por el identificador de la imagen.

In [ ]:
!ls /vagrant/indexed_storage/S2_MSI_L2A

## 3. Generar los metadatos de la imagen

___

Las imágenes que descargamos de diferentes plataformas como Copernicus o USGS Earth Explorer tienen archivos de metadatos específicos de las imágenes descargadas, pero estos metadatos no fueron generados para ser usados por herramientas como el cubo de datos. Es por ello que se requiere generar un archivo de metadatos adicional por imagen que pueda ser interpretatos por el cubo de datos. Para lo anterior, la comunidad que soporta el desarrollo del cubo de datos ha desarrollado scrips de generación de metadatos (también llamados scripts de preparación de metadatos) para generar los metadatos de imágenes de distintos sensores como Sentinel, Landsat, entre otros. 

En particular, usarémos el script de generación de metadatos para imágenes de Sentinel 2 que se encuentra disponible en el presente ambiente de desarrollo del cubo de datos, `/vagrant/indexed_storage/S2_MSI_L2A/sen2cor_prepare.py`.

Ejecute el siguiente comando para generar los metadatos de la imágen descomprimida anteriormente.

**Referencia:** Este script fue tomado de [aquí](https://github.com/dfnino10/ODC_Sentinel)

In [ ]:
# Path to the metadata generation script for sentinel 2 images
metadata_prepare_script_file_path = '/vagrant/indexed_storage/S2_MSI_L2A/sen2cor_prepare.py'
# Path to the unziped image
unziped_image_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE'
metadata_dst_file_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE'

# Execute the metadata generation script
!python3 $metadata_prepare_script_file_path  $unziped_image_path --output $metadata_dst_file_path

Compruebe con el siguiente comando si el archivo de metadatos **.SAFE.yaml** se generó para la imágen descomprimida

In [ ]:
!ls -l $unziped_image_path

Use el siguiente comando para ver los metadatos generados para la imagen descomprimida. Note que los metadatos contienen información especifica de cada imágen como la fecha en que fué capturada `datetime: '2021-01-15T17:44:59.000000Z'`, la ruta donde se encuentran las imágenes de las bandas en el sistema de archivos, entre otros. Finalmente, en los metadatos generados el script indica de forma explicita a qué producto del cubo de datos será asociada la imagen.

```sh
product:
  name: s2_sen2cor_ard_granule_EO3
```

Por supuesto, un producto con el identificador `s2_sen2cor_ard_granule_EO3` debe haber sido creado con anterioridad.

In [ ]:
# Path to the image generated metadata
image_metadata_file_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE/{image_file_name}.SAFE.yaml'

!cat $image_metadata_file_path

## 4. Adicionar la imágen al índice de imágenes del cubo de datos

___

En este punto la imágen que hemos descomprimido y a la cual hemos generado los metadatos está lista para se indexada en el cubo de datos. Para indexar una imagen en el cubo de datos lo úncio que debemos hacer es entregar al cubo de datos el archivo de metadatos de la imagen, que fue generado en el paso anterior.

El comando para indexar una imagen en el cubo de datos tiene el siguiente formato:

`datacube dataset add <image_metadata_file>.yaml`

El comando anterior se encarga de tomar los metadatos específicos de la imagen y los ingresa en la base de datos del cubo de datos. De esta forma, el cubo de datos facilita la búsqueda de imágenes.

Use el comando que está a continuación para indexar la imagen en el cubo de datos.

In [ ]:
image_metadata_file_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE/{image_file_name}.SAFE.yaml'

!datacube dataset add $image_metadata_file_path

Use el siguiente comando para compruebar que se ha indexado la imagen correctamente. Note que el cubo de datos muestra los metadatos obtenidos de la imagen. 

In [ ]:
!datacube dataset search

### References

1. [Indexing Data](https://datacube-core.readthedocs.io/en/latest/ops/indexing.html#indexing-data)